In [1]:
from io import BytesIO
import requests
import time
import pandas as pd
import schedule
import time
pd.set_option('display.max_columns', None)

API_KEY = "GVOBST0CZ3HTK3Y2"
BASE_URL = r"https://www.alphavantage.co/query?"

In [2]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][['Symbol', 'GICS Sector', 'GICS Sub-Industry']]

In [12]:
# Earnings
def earnings_history_api(symbol):
    api_key = "GVOBST0CZ3HTK3Y2"
    earn = pd.DataFrame()
    income = pd.DataFrame()
    bal = pd.DataFrame()
    
    assert symbol is not None
    symbol = symbol.strip().upper()
    
    # Earn
    url = f"{BASE_URL}function=EARNINGS&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    earn_output = response.json()
    earn = pd.DataFrame(earn_output['quarterlyEarnings'])
    earn['Symbol'] = symbol
    #earn = earn[earn['fiscalDateEnding'] > '2014-01-01']
    
    # Income
    url2 = f"{BASE_URL}function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}"
    response2 = requests.get(url2)
    income_output = response2.json()
    income = pd.DataFrame(income_output['quarterlyReports'])
    income['Symbol'] = symbol
    income = income.drop(columns=['reportedCurrency'])
    #income = income[income['fiscalDateEnding'] > '2014-01-01']
    
    # Bal
    url3 = f"{BASE_URL}function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}"
    response3 = requests.get(url3)
    bal_output = response3.json()
    bal = pd.DataFrame(bal_output['quarterlyReports'])
    bal['Symbol'] = symbol
    bal = bal.drop(columns=['reportedCurrency'])
    #bal = bal[bal['fiscalDateEnding'] > '2014-01-01']
    
    # Cashflow
    url4 = f"{BASE_URL}function=CASH_FLOW&symbol={symbol}&apikey={api_key}"
    response4 = requests.get(url4)
    cash_output = response4.json()
    cash = pd.DataFrame(cash_output['quarterlyReports'])
    cash['Symbol'] = symbol
    cash = cash.drop(columns=['reportedCurrency'])
    #cash = cash[cash['fiscalDateEnding'] > '2014-01-01']
    
    
    # Overview
    url = f"{BASE_URL}function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    over = pd.DataFrame(response.json(), index=[0])
    
    df = over.merge(earn, how='inner', on='Symbol') \
    .merge(income, how='inner', on=['Symbol', 'fiscalDateEnding']) \
    .merge(bal, how='inner', on=['Symbol', 'fiscalDateEnding']) \
    .merge(cash, how='inner', on=['Symbol', 'fiscalDateEnding'])
    
    earn = pd.DataFrame()
    income = pd.DataFrame()
    bal = pd.DataFrame()
    cash = pd.DataFrame()
    
    time.sleep(61)
    return df # earn, income, bal, over,

In [13]:
#all_companies = sp500['Symbol'].head(2).to_list()
# 'AAPL','MSFT','GOOG','GOOGL','AMZN','TSLA',
all_companies = ['FB','NVDA','BRK.B','JPM','UNH','V','JNJ','HD','WMT','PG','BAC','MA','PFE','DIS','AVGO','XOM','ACN','CSCO','NFLX']
all_companies = ['IBM']
results = pd.DataFrame()

In [14]:
for i in all_companies:
    results = results.append(earnings_history_api(i))

In [16]:
results.to_csv('IBM.csv')